1️⃣ Why Use Custom Loss Functions?

Sometimes, default loss functions may not fit well for certain problems.
For example:
Robust Regression → Custom Huber loss (less sensitive to outliers)
Imbalanced Classification → Custom weighted log loss
Ranking Problems → Custom ranking metrics
Custom functions allow us to control how the model learns based on domain knowledge.

2️⃣ Custom Objective Function (Custom Loss)

A custom objective function determines how LightGBM minimizes errors during training.
It must return:
Gradient (first derivative)
Hessian (second derivative, curvature information for optimization)
🔹 Example: Custom Hinge Loss for Classification
Hinge loss is commonly used for SVMs, but we can define it for LightGBM.

In [66]:
import numpy as np

import lightgbm as lgb

from sklearn.datasets import make_classification

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, f1_score

In [67]:
# Generate sample dataset

X, y = make_classification(n_samples=1000, n_features=10, random_state=42)


In [68]:
# Split the data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [69]:

def custom_hinge_loss(y_pred, dataset):
    
    y_true = dataset.get_label()  # Get true labels
    y_pred = 2.0 / (1.0 + np.exp(-y_pred)) - 1  # Sigmoid transformation

    grad = np.where(y_true * y_pred < 1, -y_true, 0)  # First derivative
    hess = np.ones_like(y_true)  # Second derivative (constant for hinge loss)
    
    return grad, hess


In [70]:
def f1_metric(y_pred, dataset):

    y_true = dataset.get_label()  # Get true labels

    y_pred_labels = (y_pred > 0.5).astype(int)  # Convert probabilities to 0/1
    
    score = f1_score(y_true, y_pred_labels)

    return "f1_score", score, True  # "True" because higher is better



In [71]:
# # Custom Hinge Loss Function

# def custom_hinge_loss ( y_pred, dataset):

#     y_pred = 2.0 / (1.0 + np.exp(-y_pred)) -1 # Sigmoid transformation

#     grad = np.where(y_true * y_pred < 1, -y_true, 0) # First derivative

#     hess = np.ones_like(y_true) # Second Derivative (constant for hinge loss)

#     return grad,hess

In [72]:
hinge_params = {
    "objective": custom_hinge_loss,  # Use our custom function
    "metric": "binary_error",
    
}

In [73]:
f1_params = {"objective": "binary"}


In [74]:
hinge_train_data = lgb.Dataset(X_train, label=y_train)


In [75]:
f1_train_data = lgb.Dataset(X_train, label=y_train)

f1_model = lgb.train(
    
    f1_params, 
    f1_train_data, 
    num_boost_round=100, 
    valid_sets=[f1_train_data], 
    feval=f1_metric)

[LightGBM] [Info] Number of positive: 388, number of negative: 412
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000246 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.485000 -> initscore=-0.060018
[LightGBM] [Info] Start training from score -0.060018
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

In [76]:
hinge_model = lgb.train(
    hinge_params, 
    hinge_train_data, 
    num_boost_round=100,
    # fobj=custom_hinge_loss  # Pass the fixed custom objective function
)

[LightGBM] [Info] Using self-defined objective function
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000345 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 10
[LightGBM] [Info] Using self-defined objective function
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

In [77]:
# Predict with Model

hinge_preds = hinge_model.predict(X_test)

print(f'Accuracy of the Model : {accuracy_score(y_test, hinge_preds)}')

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [ ]:
# Predict with Model

f1_preds = f1_model.predict(X_test)

print(f'Accuracy of the Model : {accuracy_score(y_test, f1_preds)}')